In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("scikit-learn-mooc/datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42
)

In [3]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1
)
preprocessor = make_column_transformer(
    (categorical_preprocessor, selector(dtype_include=object)),
    remainder="passthrough",
)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", HistGradientBoostingClassifier(random_state=42)),
    ]
)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

learning_rates = [0.01, 0.1, 1, 10]
max_leaf_nodes_list = [3, 10, 30]

best_score = -np.inf
best_params = None

for lr in learning_rates:
    for mln in max_leaf_nodes_list:
        model.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        scores = cross_val_score(model, data_train, target_train, cv=5)
        mean_score = scores.mean()

        print(f"lr={lr}, max_leaf_nodes={mln} -> CV acc={mean_score:.3f}")

        if mean_score > best_score:
            best_score = mean_score
            best_params = {"learning_rate": lr, "max_leaf_nodes": mln}

print("\nBest CV score:", round(best_score, 3))
print("Best params:", best_params)
